In [1]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import pymongo
import os

In [2]:
load_dotenv()

True

In [3]:
client = pymongo.MongoClient(os.getenv('MONGO_URI'))
db = client[os.getenv('MONGO_DATABASE')]
collection = db['scrapy_bogota_apartments']

df = pd.DataFrame(list(collection.find()))
df = df.drop(columns=['_id'], axis=1)

df.head(3)

,codigo,tipo_propiedad,tipo_operacion,precio_venta,precio_arriendo,area,habitaciones,banos,administracion,parqueaderos,...,antiguedad,estado,longitud,latitud,featured_interior,featured_exterior,featured_zona_comun,featured_sector,descripcion,datetime
0,374-M2718950,APARTAMENTO,VENTA Y ARRIENDO,2.500000e+09,13500000.0,320.0,4.0,5.0,2500000.0,3.0,...,MAS DE 20 ANOS,USADO,-74.026855,4.715475,"[ALARMA, CHIMENEA, CITOFONOS, COMEDOR AUXILIAR...","[AREA TERRAZA/BALCON 120 M2, ASCENSOR, CIRCUIT...","[CANCHA(S) DE SQUASH, CANCHA(S) DE TENNIS, GIM...","[CERCA CENTROS COMERCIALES, CERCA COLEGIOS / U...","vendo o arriendo ph duplex montearroyo, 320 mt...",2023-06-19 05:43:49.763
1,2510-M4156145,APARTAMENTO,VENTA,4.850000e+08,NaN,84.0,3.0,3.0,680000.0,1.0,...,MAS DE 20 ANOS,USADO,-74.045900,4.694500,"[CITOFONOS, TIPO DE CORTINAS NINGUNA, PARQUEAD...","[ASCENSOR, CIRCUITO CERRADO DE TV, CONJUNTO CE...","[CERCA CENTROS COMERCIALES, CERCA COLEGIOS / U...",NaN,"oportunidad, bonito apartamento, recien refacc...",2023-06-19 05:43:53.411
2,10312-M2873283,APARTAMENTO,VENTA Y ARRIENDO,1.980000e+09,9000000.0,248.0,3.0,3.0,2200000.0,3.0,...,MAS DE 20 ANOS,USADO,-74.027916,4.690854,"[APTO PARA NINOS, CHIMENEA, CITOFONOS, COMEDOR...","[AREA TERRAZA/BALCON 12 M2, ASCENSOR, ACCESO P...","[SALON COMUNAL, ZONA DE BBQ, ZONA PARA NINOS, ...","[CERCA CENTROS COMERCIALES, CERCA COLEGIOS / U...",presentamos espectacular apartamento en uno de...,2023-06-19 05:43:58.765


In [4]:
df.columns

Index(['codigo', 'tipo_propiedad', 'tipo_operacion', 'precio_venta',
       'precio_arriendo', 'area', 'habitaciones', 'banos', 'administracion',
       'parqueaderos', 'sector', 'estrato', 'antiguedad', 'estado', 'longitud',
       'latitud', 'featured_interior', 'featured_exterior',
       'featured_zona_comun', 'featured_sector', 'descripcion', 'datetime'],
      dtype='object')

In [5]:
def check_jacuzzi(x):
    if type(x) == list:
        return 1 if 'JACUZZI' in x else 0
    else:
        return 0

df['jacuzzi'] = df.featured_interior.apply(check_jacuzzi)

In [6]:
def extract_piso(x):
    if type(x) == list:
        try:
            for item in x:
                if item.startswith('PISO '):
                    return int(item.split(' ')[1])
        except:
            return np.nan
    else:
        return np.nan
    
    
df['piso'] = df.featured_interior.apply(extract_piso)

In [7]:
def extract_closets(x):
    if type(x) == list:
        try:
            for item in x:
                if item.startswith('CLOSETS'):
                    return int(item.split(' ')[1])
        except:
            return np.nan
    else:
        return np.nan
    
df['closets'] = df.featured_interior.apply(extract_closets)

In [8]:
def check_chimeny(x):
    if type(x) == list:
        return 1 if 'CHIMENEA' in x else 0
    else:
        return 0
    
df['chimenea'] = df.featured_interior.apply(check_chimeny)

In [9]:
def check_mascotas(x):
    if type(x) == list:
        return 1 if 'PERMITE MASCOTAS' in x else 0
    else:
        return 0
    
df['permite_mascotas'] = df.featured_interior.apply(check_mascotas)

In [10]:
def check_gimnasio(x):
    if type(x) == list:
        return 1 if 'GIMNASIO' in x else 0
    else:
        return 0
    
df['gimnasio'] = df.featured_zona_comun.apply(check_gimnasio)

In [11]:
def check_ascensor(x):
    if type(x) == list:
        return 1 if 'ASCENSOR' in x else 0
    else:
        return 0
    
df['ascensor'] = df.featured_exterior.apply(check_ascensor)

In [12]:
def check_conjunto_cerrado(x):
    if type(x) == list:
        return 1 if 'CONJUNTO CERRADO' in x else 0
    else:
        return 0
    
df['conjunto_cerrado'] = df.featured_exterior.apply(check_conjunto_cerrado)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26081 entries, 0 to 26080
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   codigo               26081 non-null  object        
 1   tipo_propiedad       26081 non-null  object        
 2   tipo_operacion       26081 non-null  object        
 3   precio_venta         16580 non-null  float64       
 4   precio_arriendo      10127 non-null  float64       
 5   area                 26079 non-null  float64       
 6   habitaciones         26076 non-null  float64       
 7   banos                26077 non-null  float64       
 8   administracion       20327 non-null  float64       
 9   parqueaderos         26076 non-null  float64       
 10  sector               24083 non-null  object        
 11  estrato              26074 non-null  float64       
 12  antiguedad           26068 non-null  object        
 13  estado               26081 non-

In [14]:
df = df.drop(columns=['featured_interior', 'featured_zona_comun', 'featured_exterior', 'featured_sector', 'datetime'], axis=1)
df.columns

Index(['codigo', 'tipo_propiedad', 'tipo_operacion', 'precio_venta',
       'precio_arriendo', 'area', 'habitaciones', 'banos', 'administracion',
       'parqueaderos', 'sector', 'estrato', 'antiguedad', 'estado', 'longitud',
       'latitud', 'descripcion', 'jacuzzi', 'piso', 'closets', 'chimenea',
       'permite_mascotas', 'gimnasio', 'ascensor', 'conjunto_cerrado'],
      dtype='object')

In [15]:
df.to_csv('../data/interim/apartments.csv', index=False)

In [16]:
import session_info

session_info.show()